# Lichess Tournament Games

In [ ]:
import berserk, pandas, time
from datetime import datetime
from tqdm import tqdm

# Token file in marisa's folder
with open('lichess.token') as f:
    API_TOKEN = f.read()

session = berserk.TokenSession(API_TOKEN)
client = berserk.Client(session=session)

Gets id's of tournaments either created by a top user or recently finished

In [ ]:
tournaments = []

for user in tqdm(top_users):
    time.sleep(0) # Don't overload API
    try:
        tournaments_by_user = client.tournaments.stream_by_creator(user)
        tournaments += [tournament['id'] for tournament in list(tournaments_by_user)]
    except:
        continue

latest_tournaments = client.tournaments.get()
tournaments += [t['id'] for t in list(latest_tournaments['finished'])]

Get games played during the tournaments

In [ ]:
games = []
for tournament in tqdm(tournaments):
    try:
        games_by_tournament = client.tournaments.export_games(tournament)
        for game in games_by_tournament:
            game['tournament_id'] = tournament
            games.append(game)
    except:
        continue

Convert to df

In [ ]:
games_df = pd.concat([pd.json_normalize(game) for game in games], ignore_index=True)
games_df['points1'] = games_df.apply(lambda row: 1 if row.winner == "white" else 0, axis = 1)
games_df['points2'] = games_df.apply(lambda row: 1 if row.winner == "black" else 0, axis = 1)
games_df = games_df.rename(columns={
    "createdAt": "date_time",
    "players.white.user.name": "player1_name", 
    "players.black.user.name": "player2_name"
})[["date_time", "tournament_id", "player1_name", "player2_name", "points1", "points2"]]
games_df.sort_values(by='date_time').reset_index(drop=True)

In [ ]:
games_df.to_csv('T-lichess-' + str(datetime.now()) + '.csv', index=False)